In [23]:
from __future__ import annotations
import bentoml
from datetime import datetime
from pydantic import BaseModel, PositiveInt
from typing import Optional

In [26]:
import joblib
import pickle
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib

from pprint import pprint
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_val_score
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MultiLabelBinarizer, FunctionTransformer

from sklearn import linear_model

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [6]:
class Building(BaseModel):  
    DataYear: int = 2016
    BuildingType: str
    PrimaryPropertyType: str
    SecondLargestPropertyUseType: Optional[str]
    ThirdLargestPropertyUseType: Optional[str]
    ZipCode: str
    CouncilDistrictCode: int
    Neighborhood: str
    YearBuilt: int
    NumberofBuildings: int
    NumberofFloors: int
    PropertyGFATotal: float
    PropertyGFAParking: float
    ListOfAllPropertyUseTypes: str
    LargestPropertyUseType: str
    SteamUsekBtu: float
    NaturalGastherms: float
    DefaultData: bool
    ComplianceStatus: str

In [10]:
external_data = {
    "DataYear": 2016, 
    "BuildingType": "NonResidential",
    "PrimaryPropertyType": "Other",
    "SecondLargestPropertyUseType": None,
    "ThirdLargestPropertyUseType": None,
    "ZipCode": "98101",
    "CouncilDistrictCode": "7",
    "Neighborhood": "DOWNTOWN",
    "YearBuilt": 2004,
    "NumberofBuildings": 1,
    "NumberofFloors": 11,
    "PropertyGFATotal": 299070,
    "PropertyGFAParking": 68432,
    "ListOfAllPropertyUseTypes": "Other",
    "LargestPropertyUseType": "Other",
    "SteamUsekBtu": 0,
    "NaturalGastherms": 346853.3125,
    "DefaultData": False,
    "ComplianceStatus": "Compliant"
}

building = Building(**external_data) 

In [13]:
building.model_dump()

{'DataYear': 2016,
 'BuildingType': 'NonResidential',
 'PrimaryPropertyType': 'Other',
 'SecondLargestPropertyUseType': None,
 'ThirdLargestPropertyUseType': None,
 'ZipCode': '98101',
 'CouncilDistrictCode': 7,
 'Neighborhood': 'DOWNTOWN',
 'YearBuilt': 2004,
 'NumberofBuildings': 1,
 'NumberofFloors': 11,
 'PropertyGFATotal': 299070.0,
 'PropertyGFAParking': 68432.0,
 'ListOfAllPropertyUseTypes': 'Other',
 'LargestPropertyUseType': 'Other',
 'SteamUsekBtu': 0.0,
 'NaturalGastherms': 346853.3125,
 'DefaultData': False,
 'ComplianceStatus': 'Compliant'}

In [27]:
EXAMPLE_INPUT = {
    "DataYear": 2016, 
    "BuildingType": "NonResidential",
    "PrimaryPropertyType": "Other",
    "SecondLargestPropertyUseType": None,
    "ThirdLargestPropertyUseType": None,
    "ZipCode": "98101",
    "CouncilDistrictCode": "7",
    "Neighborhood": "DOWNTOWN",
    "YearBuilt": 2004,
    "NumberofBuildings": 1,
    "NumberofFloors": 11,
    "PropertyGFATotal": 299070,
    "PropertyGFAParking": 68432,
    "ListOfAllPropertyUseTypes": "Other",
    "LargestPropertyUseType": "Other",
    "SteamUsekBtu": 0,
    "NaturalGastherms": 346853.3125,
    "DefaultData": False,
    "ComplianceStatus": "Compliant"
}


@bentoml.service
class Prediction:
    def __init__(self) -> None:
        self.model = joblib.load("model/xgb.model")
        self.pipeline = joblib.load("preprocessing.pipeline")
        self.scaler_y = joblib.load("y.scaler")

    @bentoml.api
    def summarize(self, building: Building = EXAMPLE_INPUT) -> float:
        transformed_data = self.pipeline([building])
        result = self.model.predict(transformed_data)
        return result

In [20]:
model = joblib.load("model/xgb.model")

In [22]:
pipeline = joblib.load("preprocessing.pipeline")

AttributeError: Can't get attribute 'DistrictDensityTransformer' on <module '__main__'>

In [21]:
scaler_y = joblib.load("y.scaler")